In [ ]:
# 必要なライブラリのインポート
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import chi2_contingency, pearsonr, spearmanr
import warnings
warnings.filterwarnings('ignore')

# 日本語表示の設定
plt.rcParams['font.sans-serif'] = ['Hiragino Sans', 'Yu Gothic', 'Meirio', 'Takao', 'IPAexGothic', 'IPAPGothic', 'VL PGothic', 'Noto Sans CJK JP']
plt.rcParams['axes.unicode_minus'] = False

print("ライブラリのインポートが完了しました")


In [ ]:
# データの読み込み
url = "https://raw.githubusercontent.com/npradaschnor/Pima-Indians-Diabetes-Dataset/master/diabetes.csv"
df = pd.read_csv(url)

# データの概要
print("データ形状:", df.shape)
print("\n変数一覧:")
print("- Pregnancies: 妊娠回数")
print("- Glucose: 2時間後血糖値 (mg/dL)")
print("- BloodPressure: 拡張期血圧 (mmHg)")
print("- SkinThickness: 皮下脂肪厚 (mm)")
print("- Insulin: 2時間後インスリン (μU/ml)")
print("- BMI: 体格指数")
print("- DiabetesPedigreeFunction: 遺伝的リスクスコア")
print("- Age: 年齢")
print("- Outcome: 糖尿病 (1:あり, 0:なし)")

# 最初の5行を表示
df.head()


In [ ]:
# 簡単なデータクリーニング（0を欠損値として処理）
# 生理学的にありえない0を中央値で置換
df_clean = df.copy()
zero_cols = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

for col in zero_cols:
    median_val = df_clean[df_clean[col] > 0][col].median()
    df_clean.loc[df_clean[col] == 0, col] = median_val

print("データクリーニング完了")
print(f"処理後のデータ形状: {df_clean.shape}")


In [ ]:
# 基本統計量の計算
print("=== 全体の基本統計量 ===")
print(df_clean.describe())

# 糖尿病の有無で分けた統計量
print("\n=== 健常者の統計量 ===")
print(df_clean[df_clean['Outcome'] == 0].describe()[['Glucose', 'BMI', 'Age']].round(2))

print("\n=== 糖尿病患者の統計量 ===")
print(df_clean[df_clean['Outcome'] == 1].describe()[['Glucose', 'BMI', 'Age']].round(2))


In [ ]:
# 主要変数の分布を可視化
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
key_vars = ['Glucose', 'BMI', 'Age', 'BloodPressure']

for i, var in enumerate(key_vars):
    ax = axes[i//2, i%2]
    
    # ヒストグラムとカーネル密度推定
    df_clean[df_clean['Outcome']==0][var].hist(bins=30, alpha=0.5, label='健常者', ax=ax, density=True, color='blue')
    df_clean[df_clean['Outcome']==1][var].hist(bins=30, alpha=0.5, label='糖尿病', ax=ax, density=True, color='red')
    
    # 平均値の線を追加
    mean_healthy = df_clean[df_clean['Outcome']==0][var].mean()
    mean_diabetic = df_clean[df_clean['Outcome']==1][var].mean()
    ax.axvline(mean_healthy, color='blue', linestyle='--', linewidth=2, label=f'健常者平均: {mean_healthy:.1f}')
    ax.axvline(mean_diabetic, color='red', linestyle='--', linewidth=2, label=f'糖尿病平均: {mean_diabetic:.1f}')
    
    ax.set_xlabel(var)
    ax.set_ylabel('密度')
    ax.set_title(f'{var}の分布')
    ax.legend()

plt.tight_layout()
plt.show()

print("考察: 糖尿病群では血糖値、BMI、年齢が高い傾向が見られます。")


In [ ]:
# Pearson相関係数の計算と可視化
plt.figure(figsize=(10, 8))
pearson_corr = df_clean.corr(method='pearson')
mask = np.triu(np.ones_like(pearson_corr, dtype=bool))
sns.heatmap(pearson_corr, mask=mask, annot=True, cmap='RdBu_r', center=0, 
            vmin=-1, vmax=1, square=True, linewidths=1)
plt.title('Pearson相関係数行列', fontsize=16)
plt.tight_layout()
plt.show()

# 糖尿病との相関が高い変数
outcome_corr = pearson_corr['Outcome'].sort_values(ascending=False)
print("糖尿病(Outcome)との相関係数:")
print(outcome_corr)


In [ ]:
# 特定の変数ペアの詳細な相関分析
# 例: GlucoseとBMIの関係
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# 1. 散布図（全体）
axes[0].scatter(df_clean['Glucose'], df_clean['BMI'], alpha=0.5)
axes[0].set_xlabel('Glucose (mg/dL)')
axes[0].set_ylabel('BMI')
axes[0].set_title('GlucoseとBMIの散布図（全体）')

# 回帰直線を追加
z = np.polyfit(df_clean['Glucose'], df_clean['BMI'], 1)
p = np.poly1d(z)
axes[0].plot(df_clean['Glucose'], p(df_clean['Glucose']), "r--", alpha=0.8)

# 2. 散布図（糖尿病の有無で色分け）
colors = ['blue', 'red']
labels = ['健常者', '糖尿病']
for outcome, color, label in zip([0, 1], colors, labels):
    mask = df_clean['Outcome'] == outcome
    axes[1].scatter(df_clean[mask]['Glucose'], df_clean[mask]['BMI'], 
                   alpha=0.5, color=color, label=label)
axes[1].set_xlabel('Glucose (mg/dL)')
axes[1].set_ylabel('BMI')
axes[1].set_title('GlucoseとBMIの散布図（グループ別）')
axes[1].legend()

# 3. 相関係数の計算と表示
# Pearson相関
pearson_r, pearson_p = pearsonr(df_clean['Glucose'], df_clean['BMI'])
# Spearman相関
spearman_r, spearman_p = spearmanr(df_clean['Glucose'], df_clean['BMI'])

axes[2].text(0.1, 0.8, f'Pearson相関係数: {pearson_r:.3f}', fontsize=14, transform=axes[2].transAxes)
axes[2].text(0.1, 0.7, f'p値: {pearson_p:.4f}', fontsize=12, transform=axes[2].transAxes)
axes[2].text(0.1, 0.5, f'Spearman相関係数: {spearman_r:.3f}', fontsize=14, transform=axes[2].transAxes)
axes[2].text(0.1, 0.4, f'p値: {spearman_p:.4f}', fontsize=12, transform=axes[2].transAxes)

if pearson_p < 0.05:
    axes[2].text(0.1, 0.2, '統計的に有意な相関あり', fontsize=12, color='red', transform=axes[2].transAxes)
else:
    axes[2].text(0.1, 0.2, '統計的に有意な相関なし', fontsize=12, color='blue', transform=axes[2].transAxes)

axes[2].axis('off')
axes[2].set_title('相関分析の結果')

plt.tight_layout()
plt.show()


In [ ]:
# 相関の強さを視覚的に比較
# 糖尿病との相関が強い上位5変数
top_corr_features = outcome_corr[1:6]  # Outcome自身を除く

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for i, (feature, corr_value) in enumerate(top_corr_features.items()):
    # 散布図
    for outcome, color, label in zip([0, 1], ['blue', 'red'], ['健常者', '糖尿病']):
        mask = df_clean['Outcome'] == outcome
        axes[i].scatter(df_clean[mask][feature], df_clean[mask]['Outcome'] + 
                       np.random.normal(0, 0.02, size=mask.sum()),  # ジッターを追加
                       alpha=0.3, color=color, label=label)
    
    axes[i].set_xlabel(feature)
    axes[i].set_ylabel('糖尿病（0:なし, 1:あり）')
    axes[i].set_title(f'{feature}\n相関係数: {corr_value:.3f}')
    axes[i].set_ylim(-0.5, 1.5)
    if i == 0:
        axes[i].legend()

# 最後の軸を削除
fig.delaxes(axes[5])

plt.suptitle('糖尿病との相関が強い上位5変数', fontsize=16)
plt.tight_layout()
plt.show()


In [ ]:
# 連続変数をカテゴリカル変数に変換
# 年齢をカテゴリ化
df_clean['Age_Category'] = pd.cut(df_clean['Age'], 
                                  bins=[0, 30, 40, 50, 100], 
                                  labels=['30歳未満', '30-39歳', '40-49歳', '50歳以上'])

# BMIをカテゴリ化（WHO基準）
df_clean['BMI_Category'] = pd.cut(df_clean['BMI'], 
                                  bins=[0, 18.5, 25, 30, 100], 
                                  labels=['低体重', '標準', '過体重', '肥満'])

# 血糖値をカテゴリ化
df_clean['Glucose_Category'] = pd.cut(df_clean['Glucose'], 
                                      bins=[0, 100, 125, 200], 
                                      labels=['正常', '境界型', '糖尿病型'])

print("カテゴリ変数の作成完了")


In [ ]:
# 年齢カテゴリと糖尿病の関係を検定
# クロス集計表の作成
crosstab_age = pd.crosstab(df_clean['Age_Category'], df_clean['Outcome'], 
                           margins=True, margins_name='合計')
crosstab_age.columns = ['健常者', '糖尿病', '合計']

print("=== 年齢カテゴリ × 糖尿病のクロス集計表 ===")
print(crosstab_age)
print()

# カイ二乗検定の実行
chi2, p_value, dof, expected = chi2_contingency(crosstab_age.iloc[:-1, :-1])

print(f"カイ二乗統計量: {chi2:.3f}")
print(f"p値: {p_value:.4f}")
print(f"自由度: {dof}")

if p_value < 0.05:
    print("\n結論: 年齢カテゴリと糖尿病には有意な関連がある（p < 0.05）")
else:
    print("\n結論: 年齢カテゴリと糖尿病には有意な関連がない（p ≥ 0.05）")

# 期待度数の表示
print("\n期待度数:")
expected_df = pd.DataFrame(expected, 
                          index=crosstab_age.index[:-1], 
                          columns=['健常者', '糖尿病'])
print(expected_df.round(2))


In [ ]:
# 複数のカテゴリ変数でカイ二乗検定を実行
categorical_vars = ['Age_Category', 'BMI_Category', 'Glucose_Category']
chi2_results = []

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for i, var in enumerate(categorical_vars):
    # クロス集計
    crosstab = pd.crosstab(df_clean[var], df_clean['Outcome'])
    
    # カイ二乗検定
    chi2, p_value, _, _ = chi2_contingency(crosstab)
    chi2_results.append({'変数': var, 'カイ二乗統計量': chi2, 'p値': p_value})
    
    # 視覚化（積み上げ棒グラフ）
    crosstab_pct = crosstab.div(crosstab.sum(axis=1), axis=0) * 100
    crosstab_pct.plot(kind='bar', stacked=True, ax=axes[i], 
                     color=['lightblue', 'lightcoral'])
    axes[i].set_title(f'{var}\np値: {p_value:.4f}')
    axes[i].set_ylabel('割合 (%)')
    axes[i].set_xticklabels(axes[i].get_xticklabels(), rotation=45)
    axes[i].legend(['健常者', '糖尿病'], loc='upper right')

plt.tight_layout()
plt.show()

# 結果のサマリー
chi2_summary = pd.DataFrame(chi2_results)
chi2_summary['有意性'] = chi2_summary['p値'].apply(lambda x: '***' if x < 0.001 else '**' if x < 0.01 else '*' if x < 0.05 else 'n.s.')
print("\n=== カイ二乗検定の結果まとめ ===")
print(chi2_summary)
print("\n有意水準: *** p<0.001, ** p<0.01, * p<0.05, n.s. 有意差なし")


In [ ]:
# 効果量（Cramer's V）の計算
def cramers_v(chi2, n, r, c):
    """Cramer's Vを計算する関数"""
    return np.sqrt(chi2 / (n * min(r-1, c-1)))

print("=== 効果量（Cramer's V）の計算 ===")
for var in categorical_vars:
    crosstab = pd.crosstab(df_clean[var], df_clean['Outcome'])
    chi2, _, _, _ = chi2_contingency(crosstab)
    n = crosstab.sum().sum()
    r, c = crosstab.shape
    v = cramers_v(chi2, n, r, c)
    
    print(f"\n{var}:")
    print(f"  Cramer's V = {v:.3f}")
    if v < 0.1:
        print("  効果量: ごく小さい")
    elif v < 0.3:
        print("  効果量: 小さい")
    elif v < 0.5:
        print("  効果量: 中程度")
    else:
        print("  効果量: 大きい")
